In [7]:
import os
from os.path import join as oj
import sys, time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data_pecarn
import data_psrc
import data
import train
import matplotlib.gridspec as grd

# sns.set(style="black")
# plt.style.use('dark_background')
outcome_def = 'iai_intervention' # output
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# combined

In [2]:
dummy = True
df_pecarn = data_pecarn.get_data(dummy=dummy)
feat_names, pecarn_feat_names = data.get_feat_names(df_pecarn)

df_psrc = data_psrc.get_data(dummy=dummy)
feat_names, psrc_feat_names = data.get_feat_names(df_psrc)

computing pecarn preprocessing...
computing psrc preprocessing...


In [6]:
feat_names = [k for k in psrc_feat_names if k in pecarn_feat_names]
meta = [outcome_def, 'cv_fold', 'dset']
common_feats = feat_names + meta
df1 = df_pecarn[common_feats]
df2 = df_psrc[common_feats]
df = pd.concat((df1, df2), axis=0)

# don't include doubles for binary features
feat_names = [f for f in feat_names if not f.endswith('_no')]
print(len(feat_names), feat_names)

34 ['RecodedMOI_Pedestrian/bicyclist struck by moving vehicle', 'Race_White', 'RecodedMOI_Bike collision/fall', 'RecodedMOI_Object struck abdomen', 'Race_American Indian or Alaska Native', 'VomitWretch_unknown', 'AbdomenPain_yes', 'DecrBreathSound_yes', 'VomitWretch_yes', 'InitHeartRate', 'AbdTenderDegree_Moderate', 'Race_Black or African American', 'GCSScore', 'CostalTender', 'LtCostalTender', 'AbdTrauma_yes', 'SeatBeltSign_yes', 'AbdDistention_yes', 'AbdTenderDegree_unknown', 'AbdTrauma_or_SeatBeltSign_yes', 'Hypotension', 'Race_Asian', 'AbdDistention_unknown', 'InitSysBPRange', 'RecodedMOI_unknown', 'Race_unknown', 'RecodedMOI_Motor vehicle collision', 'RecodedMOI_Fall from an elevation', 'AbdTenderDegree_Mild', 'ThoracicTrauma_yes', 'Age', 'AbdTenderDegree_Severe', 'RtCostalTender', 'RecodedMOI_Motorcycle/ATV/Scooter collision']


In [4]:
train_idxs = data.pecarn_train_idxs
test_idxs = data.psrc_train_idxs

# predict

In [ ]:
for balancing_ratio in [1, 5, 100, 1000]: 
    for balancing in ['ros', 'smote']:
         for model_type in tqdm(['logistic', 'dt', 'rf', 'mlp2', 'svm']): # 'rf', 'mlp2', 'svm', 'gb'
            for num_feats in [5, 10, 20, 34]: # 34 total feats
                feats = feat_names[:num_feats]
                out_dir = f'results/jun2_5'
                os.makedirs(out_dir, exist_ok=True)
                out_name = f'{model_type}_{len(feats)}_{balancing}={balancing_ratio}'
                train.train(df, feat_names=feat_names, model_type=model_type, 
                            balancing=balancing, outcome_def=outcome_def,
                            balancing_ratio=balancing_ratio,
                            out_name=f'{out_dir}/{out_name}.pkl',
                            train_idxs=train_idxs,
                            test_idxs=test_idxs)

/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in